Huggingface part, put this in app.py in filed pulled from huggingface by

 When prompted for a password, use an access token with write permissions.
Generate one from your settings: https://huggingface.co/settings/tokens
git clone https://huggingface.co/spaces/felixfritzen/project
Modify the files locally, then commit and push:


 git commit -am "Update space"
 
 git push

In [4]:
import pickle
import hopsworks
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

project = hopsworks.login(api_key_value="wOWQmzzHeITT5wlJ.FkOButuQ3XpDXSUO1LnTuaNfD7SPWolfF1hateoistsLFFsBF7upULn5z6qKPOIB")
fs = project.get_feature_store() 
mr = project.get_model_registry()

model = mr.get_model("KMeans_Model", version=1)
model_dir = model.download()
print(f"Model downloaded to: {model_dir}")
with open(f"{model_dir}/kmeans_model.pkl", "rb") as f:
    kmeans_model = pickle.load(f)

text_model = SentenceTransformer('all-MiniLM-L6-v2')

#encoded_headlines = model.encode(headline_df['title'].to_list())
#loaded from hopsworks!
fg_vectors = fs.get_feature_group(
    name='headlinesemb', 
    version=1,
)
retrieved_data = fg_vectors.read()
vectors = retrieved_data[[f"dim_{i}" for i in range(384)]].to_numpy()
headlines = retrieved_data['headlines']
print(vectors.shape, headlines.shape)

def df_between_dates(df, start_date, end_date):
    return df[(df['pubdate'] >= start_date) & (df['pubdate'] <= end_date)]

def get_most_similar_headlines(query_embedding, embeddings, headlines, top_n=5):
    print('compute similarities')
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(-similarities)[:top_n]
    return [(headlines[i], similarities[i]) for i in top_indices]

def compare(encoded_headlines, headlines, query, kmeans_model, text_model):
    encoding = text_model.encode([query])
    most_similar = get_most_similar_headlines(encoding, encoded_headlines, headlines, top_n=5)
    cluster = kmeans_model.predict(encoding)
    print(f"Query: {query}")
    print()
    outp = ""
    outp += str(f"Cluster: {cluster[0]}")
    outp += "\nMost Similar Headlines:"
    
    for i, (headline, similarity) in enumerate(most_similar, 1):
        outp +=f"\n{i}. {headline} (Similarity: {similarity:.4f})"
    return str(outp)
        
def apply_kmean(prompt):
    return compare(vectors, headlines, prompt, kmeans_model, text_model)

with gr.Blocks(css=".gradio-container {background-color: white;}") as demo:
    gr.Markdown("# Headline prediction app, [clustering page](https://felixfritzen.github.io/Scalable_proj/)")
    text_input = gr.Textbox(label="Enter your headline:")
    text_output = gr.Textbox(label="Most simmilar headlines:")
    gr.Button("Predict headlines").click(apply_kmean, inputs=text_input, outputs=text_output)
    gr.HTML('<iframe src="https://felixfritzen.github.io/Scalable_proj/" width="100%" height="100%" style="border:none;"></iframe>', min_height=50)

if __name__ == "__main__":
    demo.launch(share=True)

2024-12-30 10:36:56,328 INFO: HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
2024-12-30 10:36:58,008 INFO: Initializing external client
2024-12-30 10:36:58,010 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-30 10:36:59,171 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163414
Model downloaded to: /var/tmp/c02073b5-961b-447b-a93b-c843109dafb1/KMeans_Model/1
2024-12-30 10:37:01,417 INFO: Use pytorch device_name: cpu
2024-12-30 10:37:01,418 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.70s) 
(13424, 384) (13424,)
* Running on local URL:  http://127.0.0.1:7860
2024-12-30 10:37:09,006 INFO: HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-30 10:37:09,052 INFO: HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
2024-12-30 10:37:09,076 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2024-12-30 10:37:09,635 INFO: HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"
2024-12-30 10:37:09,698 INFO: HTTP Request: GET https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64 "HTTP/1.1 200 OK"
* Running on public URL: https://d02b5c6a799aed656d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
2024-12-30 10:37:11,166 IN